In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seab as sns
import numpy as np
import re
from collections import defaultdict

# 📁 Notebook's current directory
base_dir = os.getcwd()

# 🔍 Detect the "Flow Summary" folder
summary_dir = None
for folder in os.listdir(base_dir):
    if os.path.isdir(os.path.join(base_dir, folder)) and "Flow Summary" in folder:
        summary_dir = os.path.join(base_dir, folder)
        break

if not summary_dir:
    raise FileNotFoundError("❌ Could not find folder with 'Flow Summary' in the name.")

# 🧭 Mapping for case labels
case_map = {
    "A": "OO",
    "B": r"$O\tilde{T}$",
    "C": r"$\tilde{J}O$",
    "D": r"$\tilde{J}\tilde{T}$"
}
colors = ["b", "g", "r", "m"]
markers = ["o", "s", "^", "D"]

# 🎨 Styling
sns.set_theme(style="ticks", context="talk", font_scale=1.3)
sns.despine()

# 🔗 Group CSVs by (K, R, S)
grouped_files = defaultdict(list)
regex = r"Flow_Summary_K([\d.]+)_R([\d.]+)_S(\d+)_Case([A-Z])"

for file in os.listdir(summary_dir):
    if file.endswith(".csv") and file.startswith("Flow_Summary"):
        match = re.search(regex, file)
        if match:
            kappa, rate, stop, case = match.groups()
            key = (kappa, rate, stop)
            grouped_files[key].append((case, os.path.join(summary_dir, file)))

# 📊 Plotting function for combined cases
def plot_combined_cases(files_info, kappa, rate, stop, y_col, y_std_col, ylabel, short_label, output_folder):
    fig, ax1 = plt.subplots(figsize=(8, 5))

    for (case, path), color, marker in zip(sorted(files_info), colors, markers):
        df = pd.read_csv(path)
        for col in df.columns:
            if col.startswith("Mean Max") or col.startswith("Std Max"):
                df[col] /= 3000  # Normalize

        df = df.sort_values("Mean Density")
        label = case_map.get(case, case)

        ax1.errorbar(
            df["Mean Density"], df[y_col], yerr=df[y_std_col],
            fmt=marker, capsize=4, label=label, color=color, linestyle="none",
            markerfacecolor='none', markersize=8, markeredgewidth=1.5
        )

    ax1.set_xlabel(r"$\rho$", fontsize=30)
    ax1.set_ylabel(ylabel, fontsize=30)
    ax1.set_ylim(bottom=0)
    ax1.tick_params(axis='both', labelsize=20)
    ax1.grid(True)
    ax1.legend(fontsize=11, loc="upper left")

    # Save in the output folder
    filename = f"{short_label}_flow_vs_density_K{kappa}_R{rate}_S{stop}.png"
    filepath = os.path.join(output_folder, filename)
    plt.tight_layout()
    plt.savefig(filepath)
    plt.show()
    plt.close()

# 🧪 What to plot
measurements = [
    ("Mean Max Throughput", "Std Max Throughput", r"$q_{vehicle}$", "vehicle"),
    ("Mean Max Passenger Throughput", "Std Max Passenger Throughput", r"$q_{passenger}$", "passenger"),
    ("Mean Max Jeepney Throughput", "Std Max Jeepney Throughput", r"$q_{jeep}$", "jeepney"),
    ("Mean Max Truck Throughput", "Std Max Truck Throughput", r"$q_{truck}$", "truck"),
]

# 🔁 Loop through each (K, R, S) group
for (kappa, rate, stop), file_list in grouped_files.items():
    if len(file_list) < 2:
        print(f"⚠️ Skipping group with <2 cases: K={kappa}, R={rate}, S={stop}")
        continue

    # 📁 Create a folder for this parameter group
    output_folder = os.path.join(base_dir, f"K{kappa}_R{rate}_S{stop}")
    os.makedirs(output_folder, exist_ok=True)

    for y_col, y_std_col, ylabel, short_label in measurements:
        plot_combined_cases(file_list, kappa, rate, stop, y_col, y_std_col, ylabel, short_label, output_folder)

    print(f"✅ Saved plots for K={kappa}, R={rate}, S={stop} → Folder: {output_folder}")